In [363]:
import pandas as pd
import os

In [364]:
absolute_filepath = '/Users/claire/Documents/CS 122 Project Files/2010 data/'

In [365]:
drop_cols_housing = ['REGIONA','DIVISIONA','STATEA','COUNTYA','COUSUBA','PLACEA','RES_ONLYA',
             'TRUSTA','AIANHHA','AITSCEA','TTRACTA','TBLKGRPA','ANRCA','NECTAA','NECTADIVA','CNECTAA',
             'UAA','URBRURALA','CDA','SLDUA','SLDLA','ZCTA5A','SUBMCDA','SDELMA','SDSECA','SDUNIA',
             'SABINSA','METDIVA', 'CSAA', 'NAME','IFC001','IFG001','IFG002','IFG003','IFG004','IFG005','IFG006','IFG007','IFG008',
             'IFH001','IFH002','IFH003','IFH004','IFH005','IFH006','IFH007','IFH008']

drop_cols_median = ['YEAR','REGIONA','DIVISIONA','STATE','STATEA','COUNTY','COUNTYA','COUSUBA','PLACEA',
                    'TRACTA','BLKGRPA','CONCITA','AIANHHA','RES_ONLYA','TRUSTA','AITSCEA','ANRCA','CBSAA',
                    'CSAA','METDIVA','NECTAA','CNECTAA','NECTADIVA','UAA','CDCURRA','SLDUA','SLDLA','SUBMCDA',
                    'SDELMA','SDSECA','SDUNIA','PUMA5A','BTTRA','BTBGA','NAME_M', 'NAME_E', 'JTIM001']

In [366]:
housing_col_mapper = {'H7X001':'Total Pop','H7X002':'White alone','H7X003':'Black or African American alone',
                 'H7X004':'American Indian and Alaska Native alone', 'H7X005':'Asian alone',
                 'H7X006':'Native Hawaiian and Other Pacific Islander alone', 'H7X007':'Some Other Race alone',
                 'H7X008':'Two or More Races', 'IFE001':'Total Units','IFE002':'Occupied','IFE003':'Vacant',
                 'IFF001':'Total Occupied Units','IFF002':'Owned with a mortgage or a loan','IFF003':'Owned free and clear',
                 'IFF004':'Renter occupied', 'BLKGRPA': 'BLCK_GRPA', 'CONCITA':'C_CITYA'}

median_col_mapper = {'JTIE001':'Median'}

In [367]:
chicago_cbsa_code = 16980

In [368]:
# Load housing data into df
if os.path.exists(absolute_filepath + 'nhgis0012_ds172_2010_block.csv'):
    housing_df = pd.read_csv(absolute_filepath + 'nhgis0012_ds172_2010_block.csv').drop(drop_cols_housing, axis = 1)
else:
    raise IOError('This file does not exists in this location')

/Users/claire/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [369]:
# Load median data into df
if os.path.exists(absolute_filepath + 'nhgis0012_ds176_20105_2010_tract.csv'):
    median_df = pd.read_csv(absolute_filepath + 'nhgis0012_ds176_20105_2010_tract.csv', encoding = "ISO-8859-1").drop(drop_cols_median, axis = 1)
else:
    raise IOError('This file does not exists in this location')

In [370]:
# Subset on Chicago CBSA code
chi_housing_df = housing_df[housing_df['CBSAA'] == chicago_cbsa_code]
chi_housing_df.rename(columns = housing_col_mapper, inplace = True)

/Users/claire/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [371]:
# Rename columns in the median df
median_df.rename(columns = median_col_mapper, inplace = True)

In [372]:
# Shorten GISJOIN field to exclude block identifiers so that it can be used to match values
chi_housing_df['GIS_SHORT'] = chi_housing_df['GISJOIN'].str[:14]
chi_housing_df.rename(columns = {'GISJOIN': 'GISJOIN_BLOCK'}, inplace = True)

/Users/claire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/claire/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [373]:
# Merge median and raw dfs on shortened GISJOIN field
full_df = chi_housing_df.merge(median_df,left_on = ['GIS_SHORT'], right_on = ['GISJOIN'])

In [374]:
# Combine race categories
full_df['Other races'] = full_df[['American Indian and Alaska Native alone', 'Asian alone',
       'Native Hawaiian and Other Pacific Islander alone',
       'Some Other Race alone', 'Two or More Races']].sum(axis = 1)
full_df = full_df.drop(['American Indian and Alaska Native alone', 'Asian alone',
       'Native Hawaiian and Other Pacific Islander alone',
       'Some Other Race alone', 'Two or More Races'], axis = 1)

In [375]:
# Calculate race percentages
full_df['% White'] = full_df['White alone']/full_df['Total Pop']
full_df['% Black'] = full_df['Black or African American alone']/full_df['Total Pop']
full_df['% Other races'] = full_df['Other races']/full_df['Total Pop']
full_df = full_df.drop(['White alone', 'Black or African American alone','Other races'], axis = 1)

In [376]:
# Calculate unit occupancy/vacancy percentages
full_df['Owner occupied'] = full_df[['Owned with a mortgage or a loan','Owned free and clear']].sum(axis = 1)
full_df['% Occupied'] = full_df['Total Occupied Units']/full_df['Total Units']
full_df['% Vacant'] = full_df['Vacant']/full_df['Total Units']
full_df['% Owner Occupied'] = full_df['Owner occupied']/full_df['Total Occupied Units']
full_df['% Renter'] = full_df['Renter occupied']/full_df['Total Occupied Units']
full_df = full_df.drop(['Occupied', 'Vacant', 'Owned with a mortgage or a loan','Owned free and clear', 'Renter occupied'], axis = 1)

In [377]:
# Update index column name to match other years, set as index
full_df.drop(['GISJOIN', 'GIS_SHORT'], axis = 1, inplace = True)
full_df.rename(columns = {'GISJOIN_BLOCK': 'GISJOIN'}, inplace = True)
full_df.set_index(keys = 'GISJOIN', inplace = True)

In [378]:
# Reorder columns to match other years
reorder = ['YEAR', 'BLOCKA', 'BLCK_GRPA', 'TRACTA', 'C_CITYA', 'COUNTY',
       'CBSAA', 'STATE', 'Total Pop', '% White', '% Black',
       '% Other races', 'Total Units', 'Median', '% Occupied', '% Vacant',
       '% Owner Occupied', '% Renter']

full_df = full_df[reorder]

In [379]:
# Write df out to CSV
full_df.to_csv("2010_census_data.csv", sep = "|")